In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier

In [2]:
# загрузим основные данные
info_purch = pd.read_csv("train_purch.csv")

In [3]:
# в процессе работы над данными пользовался данным кодом, чтобы выявить столбцы с хотя бы одным пустым значением
'''
for i in info_purch.columns:
    summ = info_purch[i].isnull().sum()
    if summ > 0: 
        print(i, ":", summ)
'''

'\nfor i in info_purch.columns:\n    summ = info_purch[i].isnull().sum()\n    if summ > 0: \n        print(i, ":", summ)\n'

In [4]:
# по айди транзакции можно понять, где была совершена одна сессионная покупка. Многие значения совпадают, исключая product_id,  
# product_quantity и trn_sum_from_iss, trn_sum_from_red
# создадим новый фрейм для хранения данных, которые нам только предстоит проссумировать
helper = pd.DataFrame()
helper["transaction_id"] = info_purch["transaction_id"]
helper["product_id"] = info_purch["product_id"]
helper["product_quantity"] = info_purch["product_quantity"]
helper["trn_sum_from_iss"] = info_purch["trn_sum_from_iss"]
helper["trn_sum_from_red"] = info_purch["trn_sum_from_red"]

helper

,transaction_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,7e3e2e3984,9a80204f78,2.0,80.0,NaN
1,7e3e2e3984,da89ebd374,1.0,65.0,NaN
2,7e3e2e3984,0a95e1151d,1.0,24.0,NaN
3,7e3e2e3984,4055b15e4a,2.0,50.0,NaN
4,7e3e2e3984,a685f1916b,1.0,22.0,NaN
...,...,...,...,...,...
15998947,a0bb11a968,de437430e2,1.0,19.0,NaN
15998948,a0bb11a968,ad8fee4200,1.0,45.0,NaN
15998949,3b876a5532,ad8fee4200,2.0,94.0,NaN
15998950,3b876a5532,230dbb0c16,1.0,50.0,NaN


In [5]:
del info_purch["product_id"]
del info_purch["product_quantity"]
del info_purch["trn_sum_from_iss"]
del info_purch["trn_sum_from_red"]

In [6]:
# избавляюсь от копий транзакций. Потери данных не будет, так как теперь эти строки одинаковы
info_purch.drop_duplicates(subset="transaction_id", keep="first", inplace=True, ignore_index=True)

In [7]:
# возьмем час, когда совершалась покупка 
info_purch['transaction_datetime'] = pd.to_datetime(info_purch['transaction_datetime']).dt.hour

In [8]:
# начнем работу с helper. Однако преждем чем суммировать, посмотрим, что можно сделать с продукцией и как ее классифицировать
info_prod = pd.read_csv("products.csv", index_col="product_id") 

In [9]:
helper_prod = pd.DataFrame()
helper_prod["transaction_id"] = helper["transaction_id"]
helper_prod["product_id"] = helper["product_id"]

In [10]:
helper_prod = info_prod.merge(helper_prod, on="product_id")

In [11]:
# столбцы с данными успешно добавлены. Теперь можно избавиться от product_id
del helper_prod['product_id']

In [12]:
# подсчитываем кол-во для каждой транзакции, берем медианную величину
new_table = pd.DataFrame()
new_table["transaction_id"] = helper_prod["transaction_id"].unique()
for i in helper_prod.columns:
    if i in ["netto", "is_own_trademark", "is_alcohol"]:
        first = helper_prod.groupby("transaction_id")[i].median().reset_index()
        # объединяем
        new_table = first.set_index('transaction_id').merge(new_table, on="transaction_id")
# дропаем копии транзакций айди и переносим категориальные данные
helper_prod.drop_duplicates(subset="transaction_id", keep="first", inplace=True, ignore_index=True)
# таким образом мы подсчитали вес, кол-во алкоголя и собственной продукции в каждой транзакции
del helper_prod["netto"]
del helper_prod["is_own_trademark"]
del helper_prod["is_alcohol"]

In [13]:
# объединяем
helper_prod = new_table.set_index('transaction_id').merge(helper_prod, on='transaction_id')

In [14]:
# отправляем в основную таблицу 
info_purch = info_purch.set_index('transaction_id').merge(helper_prod, on='transaction_id')

In [15]:
info_purch

,transaction_id,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,is_alcohol,is_own_trademark,netto,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id
0,7e3e2e3984,000012768d,7,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0.0,0.0,0.2500,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,be9c719ecd,c7609af7c3
1,c1ca85d462,000012768d,8,5.7,0.0,0.0,0.0,574.0,ed8d2683fd,0.0,0.0,0.5000,c3d3a8e8c6,ad2b2e17d2,ca69ed9de2,3d648097f6,85.0,f0b3b7978f,634d4e126a
2,6a0e96d0bc,000012768d,10,8.0,0.0,0.0,0.0,803.0,017c89b915,0.0,0.0,0.6625,e344ab2e71,ed2ad1797c,351f8505dc,171468d7fd,21.0,8281de6bcb,6bc8b3c476
3,b34f23306e,000012768d,15,2.0,0.0,0.0,0.0,419.0,017c89b915,0.0,0.0,0.7000,c3d3a8e8c6,ad2b2e17d2,ca69ed9de2,3d648097f6,85.0,f0b3b7978f,634d4e126a
4,12b218b054,000036f903,10,1.2,0.0,0.0,0.0,241.0,6381a55c22,0.0,0.0,0.1500,c3d3a8e8c6,f2333c90fb,419bc5b424,f7c957d825,271.0,4da2dc345f,43acd80c1a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814988,7132d89b2e,fffeca6d22,9,5.0,0.0,0.0,0.0,110.0,a0613e36b5,0.0,0.0,1.0000,e344ab2e71,ed2ad1797c,6fbda2e7e2,5b8e2ec172,187.0,6c3eecd92b,cb331af8c5
2814989,83f49a713b,fffeca6d22,9,6.0,0.0,0.0,0.0,138.0,a0613e36b5,0.0,0.0,0.0200,e344ab2e71,703f4b6eb0,f4613d272f,2485f6bafd,9.0,04b308884c,4636629e05
2814990,623d3388ad,fffeca6d22,10,0.6,0.0,0.0,0.0,128.0,a0613e36b5,0.0,0.0,0.9500,c3d3a8e8c6,ad2b2e17d2,ca69ed9de2,3d648097f6,85.0,46023ca4d7,328e719854
2814991,a0bb11a968,fffeca6d22,7,0.4,0.0,0.0,0.0,99.0,a0613e36b5,0.0,0.0,0.4000,c3d3a8e8c6,ad2b2e17d2,ca69ed9de2,8bbeabc581,212.0,e0712d0d5e,3f05fc02ee


In [16]:
del helper["product_id"]

In [17]:
# проссумируем trn_sum_from_iss и product_quantity для каждой транзакции
new_table1 = pd.DataFrame()
new_table1["transaction_id"] = helper["transaction_id"].unique()
for i in helper.columns:
    if i != "transaction_id":
        first = helper.groupby("transaction_id")[i].sum().reset_index()
        # объединяем
        new_table1 = first.set_index('transaction_id').merge(new_table1, on="transaction_id")
helper = new_table1

In [18]:
# объединим helper и info_purch
info_purch = info_purch.set_index('transaction_id').merge(helper, on="transaction_id")

In [19]:
del info_purch["transaction_id"]

In [20]:
for i in info_purch.columns:
    summ = info_purch[i].isnull().sum()
    if summ > 0: 
        print(i, ":", summ)

netto : 67
level_1 : 219
level_2 : 219
level_3 : 219
level_4 : 219
segment_id : 35389
brand_id : 145732
vendor_id : 332


In [21]:
# заполним nan уникальными значениями для категорий
for i in info_purch.columns:
    summ = info_purch[i].isnull().sum()
    if summ > 0 and i != "netto": 
        info_purch[i].fillna("111111111", inplace=True)
info_purch["netto"].fillna(info_purch["netto"].median(), inplace=True)

In [22]:
info_ready = info_purch

In [23]:
# предпослений этап обработки - посмотрим на данные клиентов
info_cli = pd.read_csv("clients2.csv", index_col="client_id")

info_cli

,client_id.1,first_issue_date,first_redeem_date,age,gender
client_id,,,,,
000012768d,NaN,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
000036f903,NaN,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
00010925a5,NaN,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
0001f552b0,NaN,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
00020e7b18,NaN,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U
...,...,...,...,...,...
fffe0abb97,NaN,2017-11-27 08:56:54,2018-02-11 09:26:08,35,F
fffe0ed719,NaN,2017-09-15 08:53:24,2017-12-12 14:50:12,69,U
fffea1204c,NaN,2018-01-31 16:59:37,2018-03-12 17:02:27,73,F


In [24]:
# заполним пустые значения максимальным значением столбца
info_cli['first_redeem_date'].fillna(pd.to_datetime(info_cli['first_redeem_date']).max(), inplace=True)

In [25]:
# подглядел один из способов обработки времени
info_cli['first_issue_time'] = (pd.to_datetime(info_cli['first_issue_date']) - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
info_cli['first_redeem_time'] = (pd.to_datetime(info_cli['first_redeem_date']) - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
info_cli['issue_redeem_delay'] = info_cli['first_redeem_time'] - info_cli['first_issue_time']

In [26]:
info_cli.drop(['first_issue_date', 'first_redeem_date'], axis=1, inplace=True)

info_cli

,client_id.1,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay
client_id,,,,,,
000012768d,NaN,45,U,1501947648,1515094207,13146559
000036f903,NaN,72,F,1491832463,1492951076,1118613
00010925a5,NaN,83,U,1532449289,1536941569,4492280
0001f552b0,NaN,33,F,1498850438,1535461185,36610747
00020e7b18,NaN,73,U,1511782905,1515606605,3823700
...,...,...,...,...,...,...
fffe0abb97,NaN,35,F,1511773014,1518341168,6568154
fffe0ed719,NaN,69,U,1505465604,1513090212,7624608
fffea1204c,NaN,73,F,1517417977,1520874147,3456170


In [27]:
# удалим абсолютно пустой столбец
del info_cli["client_id.1"]

In [28]:
info_ready = info_cli.merge(info_ready, on="client_id")

In [29]:
# удалить после тестов
saver = info_ready.copy()

In [194]:
info_ready = saver.copy()

In [195]:
# отлично, пустых столбцов нет
for i in info_ready.columns:
    summ = info_ready[i].isnull().sum()
    if summ > 0: 
        print(i, ":", summ)

In [196]:
# на этом этапе нам необходимо нормализовать данные, поскольку их разброс крайне высок
# однако заметим, что _spent имеют отрицательные значения. Пропустим их через модуль 
info_ready["express_points_spent"] = abs(info_ready["express_points_spent"])
info_ready["regular_points_spent"] = abs(info_ready["regular_points_spent"])

In [197]:
# удалим все строки, которые являются выбросами
info_ready = info_ready[(info_ready["age"] <= 100) & (info_ready["age"] >= 12)]

In [198]:
# оставим записи только для каждого первого вхождения клиента в список
info_ready.drop_duplicates(subset="client_id", keep="first", inplace=True, ignore_index=True)

In [199]:
info_ready

,client_id,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,...,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,trn_sum_from_red,trn_sum_from_iss,product_quantity
0,000012768d,45,U,1501947648,1515094207,13146559,7,10.0,0.0,0.0,...,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105,be9c719ecd,c7609af7c3,0.0,1007.0,21.0
1,000036f903,72,F,1491832463,1492951076,1118613,10,1.2,0.0,0.0,...,c3d3a8e8c6,f2333c90fb,419bc5b424,f7c957d825,271,4da2dc345f,43acd80c1a,0.0,241.0,4.0
2,0001f552b0,33,F,1498850438,1535461185,36610747,7,9.1,0.0,0.0,...,c3d3a8e8c6,ad2b2e17d2,2268617da5,7e34d7a162,320,8281de6bcb,6bc8b3c476,0.0,919.0,11.0
3,00020e7b18,73,U,1511782905,1515606605,3823700,8,30.9,0.0,0.0,...,c3d3a8e8c6,ad2b2e17d2,ca69ed9de2,0736b62d31,157,1acca0f68a,f59a923b5e,0.0,2545.0,46.0
4,00022fd34f,65,U,1513359265,1525808678,12449413,14,0.8,0.0,0.0,...,e344ab2e71,ed2ad1797c,6392fb443f,63c93f54d4,126,da0c5ec76a,56f45b2e57,0.0,165.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139470,fffd5cd0c6,47,M,1498501395,1537728026,39226631,14,0.4,0.0,0.0,...,e344ab2e71,14d373dff5,1908d94f93,10fb31cfae,163,42d96f109d,9f3fceb242,0.0,83.0,1.0
139471,fffd63dfe3,31,U,1495823646,1518803995,22980349,9,1.2,0.0,0.0,...,e344ab2e71,ed2ad1797c,351f8505dc,0e680665eb,205,02acd4ebce,00a6ecb0a3,0.0,255.0,8.0
139472,fffd8c9d7d,48,F,1519134004,1530388971,11254967,9,0.5,0.0,0.0,...,c3d3a8e8c6,034aca0659,44578de904,c76a16e13e,144,4da2dc345f,e6af81215a,0.0,104.0,2.0
139473,fffe0abb97,35,F,1511773014,1518341168,6568154,7,6.0,0.0,0.0,...,e344ab2e71,1d2939ba1d,da13a5d82a,c6789438a4,148,75d8f184c7,8ea303a85f,0.0,608.0,10.0


In [200]:
# в процессе прогонки данных и их обработки, наиболее полезными оказались те, что указаны в списке. Их мы не удаляем
for i in info_ready.columns:
    if i not in ['client_id', 'age', 'first_issue_time', 'first_redeem_time', 'issue_redeem_delay', 'gender', "trn_sum_from_iss", "trn_sum_from_red"]:
        del info_ready[i]

In [201]:
# теперь вытащим категории вместе с айди клиента, перед тем, как нормализовать данные
cat_saver = pd.DataFrame()
for i in info_ready.columns: 
    if info_ready[i].dtype == 'O':
        cat_saver[i] = info_ready[i]
        if i != "client_id":
            del info_ready[i]
cat_saver["old_id"] = info_ready.index

In [202]:
# уберем айди
del info_ready["client_id"]

In [203]:
# приступим к нормализации

from sklearn import preprocessing

x = info_ready.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
norm_ready = pd.DataFrame(x_scaled)

In [204]:
# данные нормализованы. Пора приступать к последнему этапу
target = pd.read_csv("train.csv")

target

,client_id,treatment_flg,purchased
0,ad6561e2d8,1,1
1,7c1ccbf93f,1,1
2,b58fadcab6,1,1
3,e99e6fabb9,0,0
4,27fb6f8520,1,1
...,...,...,...
140022,999d284453,1,1
140023,f634deea4e,0,1
140024,16cb4f99b0,0,1
140025,23c2b72b2e,1,1


In [205]:
# проверяем на ошибки и пустые данные
print(target["treatment_flg"].unique())
print(target["purchased"].unique())
print(target["treatment_flg"].isnull().sum())
print(target["purchased"].isnull().sum())
# пустых строк нет, небинарных значений нет

[1 0]
[1 0]
0
0


In [206]:
# простым сравнением определяем, есть ли зависимость между смс и совершением покупки для каждого клиента 
target["relat"] = target["treatment_flg"] == target["purchased"]
# удаляем ненужные для нас столбцы
del target["treatment_flg"]
del target["purchased"]

target

,client_id,relat
0,ad6561e2d8,True
1,7c1ccbf93f,True
2,b58fadcab6,True
3,e99e6fabb9,True
4,27fb6f8520,True
...,...,...
140022,999d284453,True
140023,f634deea4e,False
140024,16cb4f99b0,False
140025,23c2b72b2e,True


In [207]:
cat_saver = target.set_index('client_id').merge(cat_saver, on="client_id")

In [208]:
# конечное объединение
norm_ready['old_id'] = norm_ready.index
newdata = cat_saver.set_index("old_id").merge(norm_ready, on="old_id")

In [209]:
del newdata["old_id"]

In [210]:
del newdata["client_id"]

In [211]:
copy = newdata.copy()

In [212]:
newdata = copy.copy()

In [213]:
# попробуем несколько методов, но сначала разделим выборку
target1 = newdata["relat"]
del newdata["relat"]

In [214]:
# спустя огромное число методов, пользователи Slack дали намек на то, что в моем случае лучше всего справится КэтБуст
# и это было правдой
catmodel = CatBoostClassifier(iterations=20, cat_features = ['gender']) # классификатор
catmodel.fit(newdata, target1) # обучение классификатора

Learning rate set to 0.5
0:	learn: 0.6904235	total: 70.9ms	remaining: 1.35s
1:	learn: 0.6891533	total: 124ms	remaining: 1.11s
2:	learn: 0.6891239	total: 137ms	remaining: 777ms
3:	learn: 0.6889478	total: 190ms	remaining: 760ms
4:	learn: 0.6888733	total: 261ms	remaining: 783ms
5:	learn: 0.6886837	total: 308ms	remaining: 719ms
6:	learn: 0.6885852	total: 350ms	remaining: 650ms
7:	learn: 0.6884438	total: 404ms	remaining: 606ms
8:	learn: 0.6883887	total: 479ms	remaining: 586ms
9:	learn: 0.6883842	total: 503ms	remaining: 503ms
10:	learn: 0.6882536	total: 555ms	remaining: 454ms
11:	learn: 0.6882455	total: 597ms	remaining: 398ms
12:	learn: 0.6881564	total: 648ms	remaining: 349ms
13:	learn: 0.6881246	total: 694ms	remaining: 297ms
14:	learn: 0.6880416	total: 739ms	remaining: 246ms
15:	learn: 0.6879899	total: 786ms	remaining: 196ms
16:	learn: 0.6879256	total: 831ms	remaining: 147ms
17:	learn: 0.6878260	total: 880ms	remaining: 97.7ms
18:	learn: 0.6877910	total: 953ms	remaining: 50.2ms
19:	learn: 0.

In [215]:
# посмотрим на важность признаков
imp = pd.DataFrame({
    'feature_name':newdata.columns,
    'feature_score': catmodel.feature_importances_
}).sort_values('feature_score', ascending=False).reset_index(drop=True)

imp

,feature_name,feature_score
0,2,73.068977
1,0,7.683264
2,brand_id,5.150043
3,3,5.133334
4,1,5.101835
5,gender,3.862546


In [55]:
info_purch = pd.read_csv("test_purch.csv")
info_purch

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,a375a035b9,2.0,50.0,NaN
1,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,6f9e02ee36,1.0,90.0,NaN
2,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,42e1eb8d23,1.0,20.0,NaN
3,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,5c6de3010f,2.0,85.0,NaN
4,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,343e841aaa,1.0,58.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6883733,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,24d5a54f42,1.0,164.0,NaN
6883734,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,7427d0830d,1.0,134.0,NaN
6883735,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,6c547c2ca8,1.0,60.0,NaN
6883736,fffff6ce77,48f6d115ce,2019-03-15 17:47:02,6.7,0.0,0.0,0.0,676.0,5ec64bdccb,d04e7a8338,1.0,48.0,NaN


In [96]:
newdata.drop_duplicates(subset="client_id", keep="first", inplace=True, ignore_index=True)

In [97]:
newdata = newdata.set_index("client_id")

In [98]:
out = catmodel.predict_proba(newdata)

In [99]:
anoutt = pd.DataFrame()
anoutt["client_id"] = newdata.index
anoutt["pred"] = out[:,1]

In [100]:
anoutt.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
anoutt.to_csv('out.zip', index=False,
          compression=compression_opts)